In [9]:
from Features.birds import *
from Features.sweden_map import *
from Features.weather import *


In [10]:
#golden = API_bird_data(bird_type="goleag")
#white = API_bird_data(bird_type="whteag")



In [11]:
#print(white[0])
lat = 57.7358823
lng = 13.4065944

In [12]:
map=SwedenMap()

print(map.point_to_region(lat, lng))

Gotland


In [13]:
map.middle_point_of_region("Gotland")

(57.51723947372926, 18.538432396771)

In [14]:
API_tomorrow_weather(lon=lng, lat=lat, days=7)

{'latitude': 57.732597,
 'longitude': 13.398376,
 'generationtime_ms': 0.13387203216552734,
 'utc_offset_seconds': 3600,
 'timezone': 'Europe/Berlin',
 'timezone_abbreviation': 'GMT+1',
 'elevation': 165.0,
 'daily_units': {'time': 'iso8601',
  'temperature_2m_mean': '°C',
  'precipitation_sum': 'mm',
  'visibility_mean': 'm',
  'wind_speed_10m_mean': 'km/h'},
 'daily': {'time': ['2025-12-10',
   '2025-12-11',
   '2025-12-12',
   '2025-12-13',
   '2025-12-14',
   '2025-12-15',
   '2025-12-16'],
  'temperature_2m_mean': [8.3, 8.0, 6.5, 4.5, 5.8, 6.1, 5.5],
  'precipitation_sum': [7.1, 9.8, 0.0, 5.5, 1.2, 3.6, 0.0],
  'visibility_mean': [23782.5,
   11930.0,
   19857.5,
   22687.5,
   3902.5,
   3747.5,
   29067.5],
  'wind_speed_10m_mean': [17.2, 17.8, 9.1, 13.1, 14.9, 21.8, 18.0]}}